In [ ]:
library(dplyr)

In [1]:
# org filenames: PhARscore/20230123 - phip195_PhARscores_pepsyn - max 7 iterations.rds
# etc
phar_dolphyn_194 = readRDS(paste0("../data/phip194_PhARscores_dolphyn.rds"))
phar_dolphyn_195 = readRDS(paste0("../data/phip195_PhARscores_dolphyn.rds"))
phar = rbind(phar_dolphyn_194,phar_dolphyn_195) %>% mutate(library = "dolphyn") 

phar_pepsyn_194 = readRDS(paste0("../data/phip194_PhARscores_pepsyn.rds"))
phar_pepsyn_195 = readRDS(paste0("../data/phip195_PhARscores_pepsyn.rds"))
rbind(phar_pepsyn_194,phar_pepsyn_195) %>% mutate(library = "pepsyn") %>%
rbind(phar) -> phar

phar %>%
    mutate(sample_id = gsub(sample_id,pattern = "-20A20G-", replacement= "")) %>%
    mutate(sample_id = gsub(sample_id,pattern = "-VRC-Control-VRC", replacement= "")) %>%
    mutate(plate = as.factor(substr(sample_id,1,3))) %>%
    mutate(sample = sample_id) %>%
    mutate(sample_id = gsub(str_sub(sample_id,5,-2),pattern = "-", replacement= ".")) -> 
phar

phar %>% mutate(p_val_re = 10^-p_val) -> phar

write.table("../data/PhARscores_dolphyn_pepsyn.csv", sep = ",", row.names = FALSE)

Warning message in gzfile(file, "rb"):
“cannot open compressed file '../data/phip194_PhARscores_dolphyn.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [ ]:
anno_pep = read.table("../data/PhageScan_PeptideAnno.tsv", header = TRUE, sep = "\t", row.names = 2, stringsAsFactors=FALSE)

# Hit Foldchange Data
fc194 <- read.table("../data/phipseq_0194_DolphynLa_001_Hits_foldchange.tsv", header=TRUE, row.names = 1)
fc195 <- read.table("../data/phipseq_0195_DolphynLa_001_Hits_foldchange.tsv", header=TRUE, row.names = 1)

cbind(fc194, fc195) %>% select(!contains("BEADS_ONLY")) -> fc
fc[rownames(anno_pep),] %>% log() -> lfc_org

# cleaning up sample names
anno_samples = data.frame(org = colnames(lfc_org))
anno_samples = cbind(anno_samples, str_split_fixed(anno_samples$org, "\\.",4))
anno_samples$sampleID = anno_samples$`4`
anno_samples %>% mutate(sampleID = gsub("Pre.Therapy_.", "", sampleID)) %>% mutate(sampleID = gsub(".20A20G.1", "", sampleID)) %>% 
                mutate(sampleID = gsub(".20A20G", "", sampleID)) %>% mutate(sampleID = gsub("VRC.", "", sampleID)) %>% 
                mutate(sampleID = gsub("IBD.Control.", "", sampleID)) %>% mutate(sampleID = gsub("_", "", sampleID)) -> anno_samples
colnames(lfc_org) = paste0(anno_samples$sampleID,anno_samples$`1`)

phar %>% select(sample) %>% unique() %>% sapply(convert_samplenames) %>%  as.list %>% unlist -> healthy_samples
lfc_org %>% select(all_of(healthy_samples)) %>%  write.table("../data/PhageScan_peptide_logHFC.csv", sep = ",")

In [ ]:
meta = read.table("../data/Helmsley_VRC_metadata_121721.csv", header = TRUE, sep = ",", row.names = 1)
meta[,c("Age","Sex","Race")] %>% rownames_to_column("sample_id") -> meta
phar %>% select(sample_id) %>% unique() %>% as.list %>% unlist -> healthy_samples
meta %>% subset(sample_id %in% healthy_samples) %>%  write.table("../data/PhageScan_sampleAnnotation.csv", sep = ",")